# cpd Phase scrambling

### What it this?

This script takes images from a given directory and returns their phase scrambled version. It does so using a cutoff value between low- and high-pass that is based on **cycles per degree**.

### How does it work?

Very good question. First check the following requirements:
1. Make sure this script is located in a directory (`scripts`) of which the parent directory contains a folder named `images`, where all the images you wish to work on are located.
2. Make sure the input images are 4-channel, with the background gray & transparent.
3. Input the following values in the preamble:
    - A `dva` value, indicating how big images will appear to participants during the experiment.
    - A cutoff value in `cpd` to determine the low- and high-pass threshold.

The script will operate the following steps:

0. From the input parameters, calculate the `cpi` threshold value that will be used next.
1. _Fourier transform_ images.
2. Create a _mask_ for the low- and high-pass scrambling.
3. Replace the masked values of the original phase spectrum with the _phase spectrum of random noise_, above and below this threshold for low- and high-pass scrambled images, respectively.

## Preamble

In [1]:
# give the input values here
# cutoff value in cpd
cpdcutoff = 1.5

# image size in dva
dva = 10

In [2]:
import cv2 as cv
import numpy as np
import glob
import os
import time
from numpy.random import randint
import matplotlib.pyplot as plt
np.random.seed()
print("All libraries are loaded.")

All libraries are loaded.




### Set the input path

In [3]:
# give the input path
parentPath = os.pardir
inputPath = parentPath + '/images/'

imagePath = glob.glob(inputPath + "*") # take in all the files in there
imagePath.sort() # make sure the files are sorted

# count the number of images to process
nbrIm = len(imagePath)

In [4]:
# quickly get the image size parameter
imSize = (cv.imread(imagePath[0], 0)).shape[0]

### Cutoff parameters

Based on previous literature, this script cuts off frequencies above and below `1 cpd` by default (but you can change that value, see below). With input images sized 400x400, for instance, that means scrambling above and below `8 cpi`. This value is calculated based on the image size automatically with regard to the input _cycles per degree_ value, and considering an image size of 8x8 dva (as we usually have in the lab).

Make sure to choose a cpd value to use as a cutoff. Here are some indications on how to choose:

- From a physiological point of view, the M- and P-channels are differently sensitive to frequencies above and below 1.5 cpd.
- From articles such as Canario et al 2016, values such as 1 cpd are found.

_Note that shuffling the phase of low frequencies has a much stronger effect than doing so for high frequencies._

In [5]:
# calculate the cpi cutoff value
cpicutoff = ((imSize/2)/((imSize/2)/dva)) * cpdcutoff

In [6]:
# decide on the manipulations
doLPscrambling = True
doHPscrambling = True

doManipulations = (doLPscrambling, doHPscrambling)

### Set the output path

In [7]:
# name the output paths
outputPath = parentPath + r'/output/scramblingOuput/'
outputPathLPscrambling = outputPath + 'lowPass/'
outputPathHPscrambling = outputPath + 'highPass/'

listOutputPaths = [outputPathLPscrambling, outputPathHPscrambling]

# create the necessary output folders
idx = 0

for manipulation in doManipulations:
    if manipulation:
        if not os.path.exists(listOutputPaths[idx]):
            os.makedirs(listOutputPaths[idx])
            idx +=1

## Start scrambling

The next cell runs the actual scrambling. Here is how it proceeds:
1. Two masks are created, for high- and low-pass scrambling. They mask either a central square with dimensions `cpiThesh x cpiThresh`, or everything around it.
2. The phase spectrum is extracted from the image twice.
3. A random noise image is created that has a similar pixel value distribution than the original image. The phase spectrum is obtained from that random noise.
4. Each version of the phase spectrum of the original image is amended by replacing its masked values by those of the phase spectrum of the random noise.

In [8]:
for file in imagePath:

    # read the image
    im = cv.imread(file, 0)

    # extracting names for future writing of images
    basename = os.path.basename(file)
    name = os.path.splitext(basename)[0]

    # Finding the rectangle that separates magnitude in 2 equally weighted parts

    # apply Fourier transform and get amplitude & phase
    y = np.fft.fftshift(np.fft.fft2(im)) # 2-dimensional FFT of the image
    ph_high = np.angle(y) # get the phase spectrum
    ph_low = np.angle(y) # do it twice for high- and low-pass
    mag = np.abs(y) # get the magnitude spectrum

    # Find the image center
    im_center = im.shape[0]//2, im.shape[1]//2
    
    # create the cutoff coordinates
    cutoff_box = [im_center[0] - int(cpicutoff), im_center[0] + int(cpicutoff),
                  im_center[1] - int(cpicutoff), im_center[1] + int(cpicutoff)]
    
    # here we replace parts of the phase spectrum with random noise
    # we create random noise that is normally distributed according to the mean and sd of the original image
    scale = np.std(im) # taking the SD of the phase spectrum
    loc = np.ndarray.mean(im) # taking the mean of the phase spectrum
    # get the phase spectrum of that random noise
    randomPhase = np.angle(np.fft.fft2(np.random.normal(loc = loc, scale = scale, size = im.shape)))
    
    # taking the phase center value to paste it back
    ph_center = np.angle(y)[im_center[0]-1 : im_center[0]+1, im_center[1]-1 : im_center[1]+1]
    
    # high-pass scrambling
    box_center = np.ones_like(im) # background of the mask, same size as the image
    box_center[cutoff_box[0]: cutoff_box[1], cutoff_box[2]: cutoff_box[3]] = 0
    mask_center = box_center[:]==0
    ph_high[mask_center] = randomPhase[mask_center]
    # recover the center of the phase spectrum
    ph_high[im_center[0], im_center[1]] = 0.0

    # low-pass scrambling
    box_surround = np.zeros_like(im) # background of the mask, same size as the image
    box_surround[cutoff_box[0]: cutoff_box[1], cutoff_box[2]: cutoff_box[3]] = 1
    mask_surround = box_surround[:]==0
    ph_low[mask_surround] = randomPhase[mask_surround]
    # recover the center of the phase spectrum
    ph_low[im_center[0], im_center[1]] = 0.0

    # reconstructing the high-pass version
    reconstruct_high = np.real(np.fft.ifft2(np.fft.ifftshift(np.multiply(np.abs(y), np.exp(1j * ph_high)))))
    
    # reconstructing the low-pass version
    reconstruct_low = np.real(np.fft.ifft2(np.fft.ifftshift(np.multiply(np.abs(y), np.exp(1j * ph_low)))))

    # write the images after manipulation
    cv.imwrite(outputPathLPscrambling + 'LP' + name + '.png', reconstruct_low)
    cv.imwrite(outputPathHPscrambling + 'HP' + name + '.png', reconstruct_high)